In [ ]:
!pip install torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 132.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 112.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 135.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/1

In [1]:
import zipfile
import os

# --- Replace these with your two zip filenames ---
zip1 = "/content/DiffusionGuard-main.zip"

# --- Create output folders (optional but clean) ---
out1 = "/content/main"

os.makedirs(out1, exist_ok=True)

# --- Unzip first file ---
with zipfile.ZipFile(zip1, 'r') as z:
    z.extractall(out1)

print("Done! Extracted to:", out1)


Done! Extracted to: /content/main


In [ ]:
import tarfile
import os

# --- Replace these with your two zip filenames ---
zip1 = "/content/diffusionguard_dataset.tar.gz"

# --- Create output folders (optional but clean) ---
out1 = "/content/data"

os.makedirs(out1, exist_ok=True)

# --- Unzip first file ---
with tarfile.open(zip1, "r:gz") as t:
    t.extractall(out1)

print("Done! Extracted to:", out1)


Done! Extracted to: /content/data


/tmp/ipython-input-3881585976.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out1)


In [ ]:
!pip install -r /content/main/DiffusionGuard-main/requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==2.1.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.1.1


In [ ]:
!pip install accelerate diffusers==0.27.2 transformers scipy einops omegaconf opencv-python pillow

In [1]:
%cd /content/main/DiffusionGuard-main

/content/main/DiffusionGuard-main


In [2]:
# 1. Apply the essential PGD patch (to bypass the PyTorch 2.2.0 runtime bug)
COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
print(f"Applying essential PGD patch to {COMMON_ATTACKS_PATH}...")
# This patch uses .clamp() and .to(adv.dtype) for a float-safe PGD step
!sed -i '89s/.*/        adv = adv - avg_grad.detach().clamp(min=-1.0, max=1.0).to(adv.dtype) * config.training.step_size/' {COMMON_ATTACKS_PATH}
print("Patch applied successfully.")


Applying essential PGD patch to /content/main/DiffusionGuard-main/attacks/common.py...
Patch applied successfully.


In [2]:

# 2. Run the main script
print("Running main script...")
!python main.py \
    --config-name diffusionguard \
    orig_image_name='keanu.png' \
    mask_image_names='["keanu_mask.png"]' \
    training.iters=300 \
    training.eps=16/255 \
    training.step_size=1/255

Running main script...
Traceback (most recent call last):
  File "/content/main/DiffusionGuard-main/main.py", line 3, in <module>
    from diffusers import StableDiffusionInpaintPipeline
  File "/usr/local/lib/python3.12/dist-packages/diffusers/__init__.py", line 3, in <module>
    from .configuration_utils import ConfigMixin
  File "/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py", line 34, in <module>
    from .utils import (
  File "/usr/local/lib/python3.12/dist-packages/diffusers/utils/__init__.py", line 21, in <module>
    from .accelerate_utils import apply_forward_hook
  File "/usr/local/lib/python3.12/dist-packages/diffusers/utils/accelerate_utils.py", line 24, in <module>
    import accelerate
  File "/usr/local/lib/python3.12/dist-packages/accelerate/__init__.py", line 7, in <module>
    from .accelerator import Accelerator
  File "/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py", line 27, in <module>
    from .checkpointing import 

In [6]:
# Downgrade transformers to a version compatible with huggingface-hub==0.19.0
!pip install transformers==4.28.0 --force-reinstall

  Using cached transformers-4.28.0-py3-none-any.whl.metadata (109 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadat

In [8]:
# Install the minimum required version of huggingface-hub for transformers==4.42.0
!pip install huggingface-hub==0.23.2 --force-reinstall

  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 15.1 MB/s eta 0:00:00
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
Using 

In [6]:
# Downgrade accelerate to a version compatible with huggingface-hub==0.19.0
# Version 0.14.0 is often stable with older dependency stacks.
!pip install accelerate==0.14.0 --force-reinstall

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached psutil-7.1.3-cp36-abi3-manylinux2010_x86_64.manylinux_2_12_x86_64.manylinux_2_28_x86_64.whl.metadata (23 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 11.3 MB/s eta 0:00:00
Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached packaging-25.0-py3-none-any.whl (66 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00

In [3]:
# Downgrade huggingface-hub to a version that includes the 'cached_download' function
!pip install huggingface-hub==0.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.0 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.19.0 which is incompatible.
gradio-client 1.13.3 requires huggingface-hub<2.0,>=0.19.3, but you have huggingface-hub 0.19.0 which is incompatible.
sentence-transformers 5.1.2 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.19.0 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.19.0

In [3]:
# 1. Downgrade NumPy to version 1.26.4 to satisfy the core requirement of transformers and PyTorch
!pip install numpy==1.26.4

# 2. Re-install transformers to ensure it recognizes the newly downgraded NumPy
# transformers 4.42.0 is needed for sentence-transformers, so we force-reinstall it here.
!pip install transformers==4.42.0 --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.8.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.8.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.29.1 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 wh

  Using cached transformers-4.42.0-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached typing_extensi

In [7]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 102.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you hav

In [1]:
# Install PyTorch 2.2.0 (the closest available version to the required 2.1.1)
!pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [4]:
# 1. Install the missing packages using the file that excluded the problematic torch==2.1.1
# This assumes you still have 'requirements_no_torch.txt' from earlier steps.
!pip install -r requirements.txt

# 2. Install the necessary, compatible version of PyTorch separately.
# This version is the closest available one, despite the known runtime error.
!pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 3. Apply the ultimate patch immediately after the setup,
# as this patch is necessary to try and bypass the runtime error in PyTorch 2.2.0.
COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
!sed -i '89s/.*/        adv = adv - avg_grad.detach().clamp(min=-1.0, max=1.0).to(adv.dtype) * config.training.step_size/' {COMMON_ATTACKS_PATH}

ERROR: Could not find a version that satisfies the requirement torch==2.1.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.1.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 132.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# Apply the ultimate PGD patch (to bypass the PyTorch 2.2.0 bug)
COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
print(f"Applying essential PGD patch to {COMMON_ATTACKS_PATH}...")
# This patch uses .clamp() and .to(adv.dtype) for a float-safe PGD step
!sed -i '89s/.*/        adv = adv - avg_grad.detach().clamp(min=-1.0, max=1.0).to(adv.dtype) * config.training.step_size/' {COMMON_ATTACKS_PATH}
print("Patch applied successfully.")

Applying essential PGD patch to /content/main/DiffusionGuard-main/attacks/common.py...
Patch applied successfully.


In [6]:
# Upgrade transformers to a version compatible with sentence-transformers (>=4.41.0)
!pip install transformers==4.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstalling transformers-4.36.0:
      Successfully uninstalled transformers-4.36.0


In [ ]:
# Downgrade NumPy to prevent crashes
!pip install numpy==1.26.4

In [5]:
# 3. Apply the ultimate patch immediately after the setup,
# as this patch is necessary to try and bypass the runtime error in PyTorch 2.2.0.
COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
!sed -i '89s/.*/        adv = adv - avg_grad.detach().clamp(min=-1.0, max=1.0).to(adv.dtype) * config.training.step_size/' {COMMON_ATTACKS_PATH}

In [9]:
# Downgrade diffusers to a version that does not call the missing internal PyTorch function.
!pip install diffusers==0.19.0 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.5/960.5 kB 21.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.0/516.0 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 16.2 M

In [2]:
# Downgrade NumPy to stabilize PyTorch and transformers (fixes multiple conflicts)
!pip install numpy==1.26.4

# Downgrade huggingface-hub to fix the 'hf_cache_home' ImportError and transformers conflict
!pip install huggingface-hub==0.25.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.5
    Uninstalling huggingface_hub-1.1.5:
      Successfully uninstalled huggingface_hub-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.25.0 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [ ]:
# Downgrade huggingface-hub to fix the 'hf_cache_home' ImportError
!pip install huggingface-hub==0.25.0

In [3]:
# Fix fsspec conflict (datasets and gcsfs dependencies)
!pip install fsspec==2025.3.0

# Fix requests conflict (google-colab dependency)
!pip install requests==2.32.4

# Fix Pillow conflict (gradio dependency)
!pip install Pillow==11.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.25.0 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [7]:
# Install the oldest available compatible versions of JAX and JAXlib
!pip install jax==0.4.17 jaxlib==0.4.17

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.11.28 requires jax>=0.6.0, but you have jax 0.4.17 which is incompatible.
chex 0.1.90 requires jax>=0.4.27, but you have jax 0.4.17 which is incompatible.
chex 0.1.90 requires jaxlib>=0.4.27, but you have jaxlib 0.4.17 which is incompatible.
optax 0.2.6 requires jax>=0.5.3, but you have jax 0.4.17 which is incompatible.
optax 0.2.6 requires jaxlib>=0.5.3, b

In [9]:
# Downgrade flax to a version compatible with jax==0.4.17
!pip install flax==0.7.0 --force-reinstall

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.5 MB/s eta 0:00:00
Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (807 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 24.8 MB/s eta 0:00:00
Using cached typing_extensions-4.15.

In [8]:
# Run the main script
!python main.py \
    --config-name diffusionguard \
    orig_image_name='keanu.png' \
    mask_image_names='["keanu_mask.png"]' \
    training.iters=300 \
    training.eps=16/255 \
    training.step_size=1/255

Traceback (most recent call last):
  File "/content/main/DiffusionGuard-main/main.py", line 3, in <module>
    from diffusers import StableDiffusionInpaintPipeline
  File "/usr/local/lib/python3.12/dist-packages/diffusers/__init__.py", line 38, in <module>
    from .models import (
  File "/usr/local/lib/python3.12/dist-packages/diffusers/models/__init__.py", line 35, in <module>
    from .controlnet_flax import FlaxControlNetModel
  File "/usr/local/lib/python3.12/dist-packages/diffusers/models/controlnet_flax.py", line 16, in <module>
    import flax
  File "/usr/local/lib/python3.12/dist-packages/flax/__init__.py", line 24, in <module>
    from flax import core
  File "/usr/local/lib/python3.12/dist-packages/flax/core/__init__.py", line 24, in <module>
    from .lift import (
  File "/usr/local/lib/python3.12/dist-packages/flax/core/lift.py", line 27, in <module>
    from flax import traverse_util
  File "/usr/local/lib/python3.12/dist-packages/flax/traverse_util.py", line 66, in <m

In [ ]:
# Check current directory (should be /content/main/DiffusionGuard-main)
!pwd

# List contents to confirm main.py and assets folder exist
!ls

/content/main/DiffusionGuard-main
assets	attacks  config  main.py  readme.md  requirements.txt  utils


In [3]:
# Downgrade NumPy to a version below 2 (e.g., 1.26.4) to stabilize the environment
!pip install numpy==1.26.4

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.0 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 1.1.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, 

In [2]:
!python main.py \
    --config-name diffusionguard \
    orig_image_name='keanu.png' \
    mask_image_names='["keanu_mask.png"]' \
    training.iters=300 \
    training.eps=16/255 \
    training.step_size=1/255


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/main/DiffusionGuard-main/main.py", line 2, in <module>
    import torch
  File "/usr/local/lib/python3.12/dist-packages/torch/__init__.py", line 1471, in <module>
    from .functional import *  # noqa: F403
  File "/usr/local/lib/python3.12/dist-packages/torch/functional.py", line 9, in <module>
    import torch.nn.functional as F
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/__init__.py", line 1, in <module>
    from .modules import *  # noqa: F403
  File "/usr/local/lib/python3.12/dist-package

In [ ]:
import fileinput
import sys
import os

COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
print(f"Patching {COMMON_ATTACKS_PATH} just before execution...")

# --- Apply the Patch ---
try:
    with fileinput.FileInput(COMMON_ATTACKS_PATH, inplace=True) as file:
        for line in file:
            # Find the line that performs the PGD update
            if 'adv = adv - avg_grad.detach().sign() * config.training.step_size' in line:
                # Replace it with the patched version using .float()
                # We need to use raw strings or escape the backslashes if done manually,
                # but fileinput handles the context.
                patched_line = line.replace('.sign()', '.sign().float()')
                sys.stdout.write(patched_line)
            else:
                sys.stdout.write(line)
    print("Patch applied successfully. Starting optimization...")
except Exception as e:
    print(f"An error occurred during patching: {e}. Cannot continue.")
    raise

# --- Run the Main Script ---
# This executes the optimization process (300 iterations)
!python main.py \
    --config-name diffusionguard \
    orig_image_name='keanu.png' \
    mask_image_names='["keanu_mask.png"]' \
    training.iters=300 \
    training.eps=16/255 \
    training.step_size=1/255

Patching /content/main/DiffusionGuard-main/attacks/common.py just before execution...
Patch applied successfully. Starting optimization...
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2025-11-25 17:15:06.412225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764090906.432393   21466 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764090906.438523   21466 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764090906.454718   21466 computation_p

In [ ]:
# 1. Define the file path
COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'

# 2. Use 'sed' to replace the PGD update line entirely with a float-safe PGD step.
# Replace the old PGD line (line 89) with a command that ensures a float-safe sign operation.
print(f"Applying ultimate shell patch to {COMMON_ATTACKS_PATH}...")
# The sed command will target the line and use torch.clamp for the sign function
!sed -i '89s/.*/        adv = adv - avg_grad.detach().clamp(min=-1.0, max=1.0).to(adv.dtype) * config.training.step_size/' {COMMON_ATTACKS_PATH}
print("Ultimate patch applied successfully with sed. Starting optimization...")

# 3. Run the main script
!python main.py \
    --config-name diffusionguard \
    orig_image_name='keanu.png' \
    mask_image_names='["keanu_mask.png"]' \
    training.iters=300 \
    training.eps=16/255 \
    training.step_size=1/255

Applying ultimate shell patch to /content/main/DiffusionGuard-main/attacks/common.py...
Ultimate patch applied successfully with sed. Starting optimization...
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2025-11-25 17:21:31.064676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764091291.084756   23153 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764091291.091207   23153 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764091291.106945  

In [ ]:
import fileinput
import sys

COMMON_ATTACKS_PATH = '/content/main/DiffusionGuard-main/attacks/common.py'
print(f"Patching {COMMON_ATTACKS_PATH}...")

try:
    with fileinput.FileInput(COMMON_ATTACKS_PATH, inplace=True) as file:
        for line in file:
            # Find the line that performs the PGD update
            if 'adv = adv - avg_grad.detach().sign() * config.training.step_size' in line:
                # Replace it with the patched version using .float()
                patched_line = line.replace('.sign()', '.sign().float()')
                sys.stdout.write(patched_line)
            else:
                sys.stdout.write(line)
    print("Patch applied successfully. PGD update line modified.")
except Exception as e:
    print(f"An error occurred during patching: {e}")

Patching /content/main/DiffusionGuard-main/attacks/common.py...
Patch applied successfully. PGD update line modified.


In [ ]:
# Downgrade peft to a more stable older version
!pip install peft==0.11.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.18.0
    Uninstalling peft-0.18.0:
      Successfully uninstalled peft-0.18.0


In [ ]:
# Install the first available compatible version of PyTorch (2.2.0)
# Note: This is necessary because 2.1.1 is not available.
!pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
# Assuming you are still in the /content/main/DiffusionGuard-main directory
!pip install -r requirements.txt

  Using cached diffusers-0.24.0-py3-none-any.whl.metadata (18 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.1.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.1.1


In [ ]:
!pip show hydra-core

Name: hydra-core
Version: 1.3.2
Summary: A framework for elegantly configuring complex applications
Home-page: https://github.com/facebookresearch/hydra
Author: Omry Yadan
Author-email: omry@fb.com
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: antlr4-python3-runtime, omegaconf, packaging
Required-by: 


In [ ]:
# Install a recent, compatible version of the transformers library
# This should resolve the 'CLIPTextModel' import error.
!pip install transformers==4.36.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.


In [ ]:
# Upgrade transformers to a version compatible with sentence-transformers (>=4.41.0)
!pip install transformers==4.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 113.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstalling transformers-4.36.0:
      Successfully uninstalled transformers-4.36.0


In [ ]:
# Explicitly install hydra-core and its necessary components
!pip install hydra-core omegaconf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Downgrade NumPy to a version below 2 (e.g., 1.26.4)
!pip install numpy==1.26.4

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompa

In [ ]:
# Install a different version of diffusers (e.g., 0.22.0) to avoid the SyntaxError
!pip install diffusers==0.22.0 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.0/516.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Re-install the correct version of huggingface-hub to fix potential dependency breakage
!pip install huggingface-hub==0.25.0

  Using cached huggingface_hub-0.25.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.25.0-py3-none-any.whl (436 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.5
    Uninstalling huggingface_hub-1.1.5:
      Successfully uninstalled huggingface_hub-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.0 requires numpy<2.0,>=1.17, but you have numpy 2.3.5 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.25.0 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
